In [ ]:
!pwd
!pip install --upgrade pip

# Install required libraries
!python3 -m pip -q install redis
!pip install -U langchain gradio
!pip install langchain-community "unstructured[pdf]"
!pip install -U langchain-google-vertexai


In [ ]:
## Uncomment & execute the following code in case Redis Enterprise is not available
##################################################################################

# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes

In [1]:
## Update the 'host' field with the correct Redis host URL
host = ''
port =
password = ''
requirePass = True

## For redis-stack-server, comment out the above code and uncomment the following:
# host = 'localhost'
# requirePass = False

In [2]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)

print(client.ping())
# Clear Redis database (optional)
client.flushdb()

REDIS_URL = f"redis://:{password}@{host}:{port}"
INDEX_NAME = f"idx_qna"

True


In [ ]:
## Authenticate with GCP & set project id and region
from google.colab import auth
from getpass import getpass

auth.authenticate_user()
print('Authenticated')


In [ ]:
# fetch your GCP project ID and region for Vertex AI
import os

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')

In [ ]:
!wget https://storage.googleapis.com/abhi-data-2024/how_india_shops_online.pdf -O report.pdf


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA

from langchain_community.document_loaders import UnstructuredPDFLoader


doc = "report.pdf"

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=100, add_start_index=True
)
loader = UnstructuredPDFLoader(doc)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)
print("Preprocessing completed and created", len(chunks), "chunks of the original pdf", doc)


Preprocessing completed and created 42 chunks of the original pdf report.pdf


# Create text embeddings with Vertex AI embedding model

Use the Vertex AI API for text embeddings, developed by Google.

Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:


*   Semantic search: Search text ranked by semantic similarity.
*   Recommendation: Return items with text attributes similar to the given text.
*   Classification: Return the class of items whose text attributes are similar to the given text.
*   Clustering: Cluster items whose text attributes are similar to the given text.
*   Outlier Detection: Return items where text attributes are least related to the given text.

The Vertex AI text-embeddings API lets you create a text embedding using Generative AI on Vertex AI. The textembedding-gecko model accepts a maximum of 3,072 input tokens (i.e. words) and outputs 768-dimensional vector embeddings.

In [5]:
from langchain.vectorstores.redis import Redis
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains import RetrievalQA
from langchain_google_vertexai import VertexAIEmbeddings

from langchain.document_loaders import UnstructuredFileLoader

embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003", project=PROJECT_ID, location=REGION)

def get_vectordb() -> Redis:
    """Create the Redis vectordb."""

    try:
        vectordb = Redis.from_existing_index(
            embedding=embeddings,
            index_name=INDEX_NAME,
            redis_url=REDIS_URL
        )
        return vectordb
    except:
        pass

    # Load Redis with documents
    vectordb = Redis.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=INDEX_NAME,
        redis_url=REDIS_URL
    )
    return vectordb


redis = get_vectordb()

#embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Include RAG

We're going to build a complete RAG pipeline from scratch incorporating the following components:

Standard retrieval and chat completion
Dense content representation to improve accuracy
Query re-writing to improve accuracy
Semantic caching to improve performance
Conversational session history to improve personalization

### Define Prompt template
PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [16]:
#@title Function to define prompt template

def create_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate
    from langchain.chains import RetrievalQA

    # Define our prompt
    prompt_template = """Use only the following pieces of context to answer the question. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt


In [17]:
#@title Invoke Google Vertex LLM using Langchain
# This is where the Langchain brings all the components together in a form of a simple QnA chain
from langchain_google_vertexai import VertexAI

llm = VertexAI(
    model_name="gemini-1.5-pro-preview-0409",
    max_output_tokens=2048,
    temperature=0.5,
    verbose=False,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=redis.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.5}),
    #return_source_documents=True,
    chain_type_kwargs={"prompt": create_prompt()},
    #verbose=True
    )

In [10]:
qa.run('What are some motivations for shopping online?')

'Question: What are some motivations for shopping online?\nAnswer: Some motivations for shopping online include wider range of price options, wider range of brands available, the absence of physical stores for premium brands, stockouts of certain products, a lack of knowledgeable staff in offline stores, lack of discounts and special offers in physical stores, and large crowds in malls during weekends. \n'

In [25]:
qa.run('How do Indians like to pay for shopping online?')

"Question: How do Indians like to pay for shopping online?\nAnswer: Indians both in urban and other areas are increasingly using UPI payments. However, cash on delivery is still preferred, especially outside of major cities.  Many people use cards for higher-priced items on familiar sites because they feel it's more secure. \n"

In [19]:
qa.run('What are some known challenges in shopping online?')

'Question: What are some known challenges in shopping online?\nAnswer: Some known challenges of shopping online include worrying about online payments and if products will arrive, not being sure what a product will look like in person based on a picture, and longer delivery times. \n'

In [26]:
qa.run('How home and kitchen segment is growing?')

'Question: How home and kitchen segment is growing?\nAnswer: In the Indian market, the home furnishing and kitchen sector has experienced remarkable growth and is growing at a CAGR of 10%. Several factors contribute to this growth including the country’s urbanisation, a substantial young population and the increasing aspirations of the burgeoning middle class. \n'

In [23]:
qa.run('What are the effects of social media on online shopping?')

'Question: What are the effects of social media on online shopping?\nAnswer: Social media has amplified the awareness and aspirations and 62% of respondents tried products after seeing them (perhaps repeatedly) on Facebook and Instagram. As the preferences of rest of India shifts from TV and radio to digital avenues, social media becomes the most preferred channel for encouraging trials of new products. Interestingly, amongst urban dwellers, online communities lead to more trial than TV. WhatsApp, YouTube and Instagram have a universal appeal and are the most preferred social media platforms for urban dwellers and rest of India. \n'

In [27]:
qa.run('What are some relevant items that are shopped online?')

'Question: What are some relevant items that are shopped online?\nAnswer: The text lists fashion and accessories, home and kitchen, beauty and personal care, health and wellness, sports and fitness, electronics and consumer durables, and groceries. \n'

In [ ]:
import gradio as gr

def handle(query):
    response = qa.run(query)
    return response

iface = gr.Interface(fn=handle, inputs="text", outputs="text")
iface.launch(share=True)

In [28]:
iface.close()

Closing server running on port: 7860
